## Setting up model

In [1]:
import spacy
import pandas as pd

from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_pickle("data.pkl")
data = data[['text','class']]

In [3]:
data.info()
class_list = data['class'].unique()
print(class_list)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5362 entries, 0 to 5361
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5362 non-null   object
 1   class   5362 non-null   object
dtypes: object(2)
memory usage: 83.9+ KB
['neutral' 'bad' 'blocker' 'good']


## Splitting data set

For now, we will just train a model to predict the "point" attribute for a given sentence.

In [4]:
Train, Valid = train_test_split(data,test_size = 0.2)

Train = list(Train.to_records(index=False))
Valid = list(Valid.to_records(index=False))

In [5]:
# print(Train)

[('<br>Last updated: August 29, 2014</p>', 'neutral'), ('We collect personal information in the following ways:\n<ul>\n<li>When you sign up to online actions or express an interest in joining us as an international member</li>\n<li>When you make a donation to us</li>\n<li>When you sign up for our human rights education courses</li>\n<li>When you participate in other online campaigns or activities for example Amnesty Decoders</li>\n<li>When you apply for a job or volunteering opportunity with us', 'good'), ('When you register a GOG account with us, you will need to provide us: (a) your username.\n(b) your email address.\nand (c) a password (which is encrypted so that we don’t have access to it).', 'good'), ('You can ask us or third parties to stop sending you marketing messages at any time by logging into the website and checking or unchecking relevant boxes to adjust your marketing preferences or by following the opt-out links on any marketing message sent to you or by Contacting us at

## Writing out to binary file

In [6]:
def make_docs(data):
    docs = []
    for doc, ann in nlp.pipe(data,as_tuples=True):
                
        if ann == "good":
            doc.cats['A'] = 0
            doc.cats['B'] = 0
        elif ann == "neutral":
            doc.cats['A'] = 1
            doc.cats['B'] = 0
        elif ann == "bad":
            doc.cats['A'] = 0
            doc.cats['B'] = 1
        elif ann == "blocker":
            doc.cats['A'] = 1
            doc.cats['B'] = 1
        else:
            print("Error")
        
        docs.append(doc)
    return (docs)

In [7]:
nlp = spacy.load("en_core_web_sm")

In [8]:
train_docs = make_docs(Train)
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("./data/train.spacy")

valid_docs = make_docs(Valid)
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("./data/valid.spacy")